In [1]:
import requests
from bs4 import BeautifulSoup
import re
import csv
from time import sleep
from random import randint
import pandas as pd
import json

In [2]:
# attention to make the api available on other computers
# jupyter kernelgateway --generate-config
# /Users/laetitiahoquetis/.jupyter/jupyter_kernel_gateway_config.py
# If you want your api to be accessible from other computers, edit the ~/.jupyter/jupyter_kernel_gateway_config.py file. Replace
# # c.KernelGatewayApp.ip = '127.0.0.1'
# with
# c.KernelGatewayApp.ip = '*'

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [4]:
# url = 'https://www.gamespot.com/games/metroid-prime-4/'
# result = requests.get(url, headers=headers)
# soup = BeautifulSoup(result.content, "html.parser")
# #print(page.content.decode())

In [5]:
# same for https://www.gameinformer.com/2023
# inside div then <span class="calendar_entry">
# get name for s href link
# <a href="/product/a-space-for-the-unbound" hreflang="en">A Space for the Unbound</a>
# (<em>PlayStation 5, Xbox Series X/S, PlayStation 4, Xbox One, Switch, PC</em>)		– <time datetime="00Z" class="datetime">January 19</time>
#  </span>
# then platform m
# <em>PlayStation 5, Xbox Series X/S, PlayStation 4, Xbox One, Switch, PC</em>
# then date 
# <time datetime="00Z" class="datetime">January 19</time>

In [6]:
pages = ['', 'xbox-one', 'ps4', '3ds', 'pc', 'nintendo-switch']
event = []
date = []

In [ ]:
for page in pages: 
    url = f"https://www.gamespot.com/gamespot-50/{page}"
    print(url)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    titles = soup.find_all("h3")
    for t in titles:
        try:
            event.append(t.a.text.strip())
        except Exception:
            event.append('n/a')
    dates = soup.find_all(class_="media-release-date")
    for d in dates:
        date.append(d.findChild().text)
    print("page parsed")
    sleep(randint(2,20))
    

https://www.gamespot.com/gamespot-50/
page parsed
https://www.gamespot.com/gamespot-50/xbox-one
page parsed
https://www.gamespot.com/gamespot-50/ps4
page parsed
https://www.gamespot.com/gamespot-50/3ds
page parsed
https://www.gamespot.com/gamespot-50/pc
page parsed
https://www.gamespot.com/gamespot-50/nintendo-switch
page parsed


In [ ]:
# titles = soup.find_all("h3")
# for t in titles:
#     try:
#         event.append(t.a.text.strip())
#     except Exception:
#         event.append('n/a')
    
# dates = soup.find_all(class_="media-release-date")
# for d in dates:
#     date.append(d.findChild().text)

In [ ]:
pd.set_option('display.max_rows', 500)
df = pd.DataFrame(list(zip(event,date)),
               columns =['event', 'date'])
df = df.drop_duplicates(subset=['event'])
df = df[df['event']!= "n/a"]
df

,event,date
0,Resident Evil 4,"Mar 24, 2023"
1,The Legend of Zelda: Tears of the Kingdom,"May 12, 2023"
2,Diablo IV,"Jun 6, 2023"
3,Final Fantasy XVI,"Jun 22, 2023"
4,Star Wars Jedi: Survivor,"Apr 28, 2023"
5,Starfield,"Sep 6, 2023"
6,Street Fighter 6,"Jun 2, 2023"
7,Dead Island 2,"Apr 21, 2023"
8,Dragon Age: Dreadwolf,Unknown Release Date
9,Suicide Squad: Kill The Justice League,"May 26, 2023"


In [ ]:
df['paged'] = df['event'].astype(str)
df['paged'] = df['paged'].str.lower()
df['paged'] = df['paged'].str.replace(":","").str.replace("'","").str.replace(".","").str.replace("&","and").str.replace("- ","")
df['paged'] = df['paged'].str.replace(" ","-")  
df['paged'] = df['paged'].apply(lambda x: x.replace(',', ''))
df['paged'] = df['paged'].apply(lambda x: x.replace('!', ''))

/var/folders/jt/qpx02c196772_ytczgpv4llh0000gp/T/ipykernel_60207/4213643871.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['paged'] = df['paged'].str.replace(":","").str.replace("'","").str.replace(".","").str.replace("&","and").str.replace("- ","")


In [ ]:
# df.event needs to be modified into paged list where space replaced by "-"
# also lower cased and remove when any ": or ' or . or - " or ','
# then serach for platform = []
# then search for des = []

In [ ]:
# # #url = f'https://www.gamespot.com/games/'{df.event}
# url = 'https://www.gamespot.com/games/resident-evil-4/'
# result = requests.get(url, headers=headers)
# soup = BeautifulSoup(result.content, "html.parser")
# #print(result.content.decode())
# #ul class = game-module__platform
# # #p class = game-module__description

In [ ]:
# res = soup.find('ul', attrs={'class' : 'game-module__platform'}).get_text()
# type(res)

In [ ]:
# soup.find('p', attrs={'class' : 'game-module__description'}).get_text()

In [ ]:
platform = []
des = []

In [ ]:
df

,event,date,paged
0,Resident Evil 4,"Mar 24, 2023",resident-evil-4
1,The Legend of Zelda: Tears of the Kingdom,"May 12, 2023",the-legend-of-zelda-tears-of-the-kingdom
2,Diablo IV,"Jun 6, 2023",diablo-iv
3,Final Fantasy XVI,"Jun 22, 2023",final-fantasy-xvi
4,Star Wars Jedi: Survivor,"Apr 28, 2023",star-wars-jedi-survivor
5,Starfield,"Sep 6, 2023",starfield
6,Street Fighter 6,"Jun 2, 2023",street-fighter-6
7,Dead Island 2,"Apr 21, 2023",dead-island-2
8,Dragon Age: Dreadwolf,Unknown Release Date,dragon-age-dreadwolf
9,Suicide Squad: Kill The Justice League,"May 26, 2023",suicide-squad-kill-the-justice-league


In [ ]:
for page in df.paged: 
    url = f"https://www.gamespot.com/games/{page}"
    print(url)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    res = soup.find('ul', attrs={'class' : 'game-module__platform'}).get_text()
    platform.append(res)
    desc = soup.find('p', attrs={'class' : 'game-module__description'}).get_text()
    des.append(desc)
    print("page parsed")
    sleep(randint(2,10))
    

https://www.gamespot.com/games/resident-evil-4
page parsed
https://www.gamespot.com/games/the-legend-of-zelda-tears-of-the-kingdom
page parsed
https://www.gamespot.com/games/diablo-iv
page parsed
https://www.gamespot.com/games/final-fantasy-xvi
page parsed
https://www.gamespot.com/games/star-wars-jedi-survivor
page parsed
https://www.gamespot.com/games/starfield
page parsed
https://www.gamespot.com/games/street-fighter-6
page parsed
https://www.gamespot.com/games/dead-island-2
page parsed
https://www.gamespot.com/games/dragon-age-dreadwolf
page parsed
https://www.gamespot.com/games/suicide-squad-kill-the-justice-league
page parsed
https://www.gamespot.com/games/bayonetta-origins-cereza-and-the-lost-demon
page parsed
https://www.gamespot.com/games/redfall
page parsed
https://www.gamespot.com/games/assassins-creed-mirage
page parsed
https://www.gamespot.com/games/the-lord-of-the-rings-gollum
page parsed
https://www.gamespot.com/games/minecraft-legends
page parsed
https://www.gamespot.com

In [ ]:
df.head(3)

,event,date,paged
0,Resident Evil 4,"Mar 24, 2023",resident-evil-4
1,The Legend of Zelda: Tears of the Kingdom,"May 12, 2023",the-legend-of-zelda-tears-of-the-kingdom
2,Diablo IV,"Jun 6, 2023",diablo-iv


In [ ]:
df['platform'] = platform
df['des'] = des

In [ ]:
df.sample(5)

,event,date,paged,platform,des
0,Resident Evil 4,"Mar 24, 2023",resident-evil-4,PCPlayStation 4+ 3 morePlayStation 5Xbox OneXb...,
4,Star Wars Jedi: Survivor,"Apr 28, 2023",star-wars-jedi-survivor,PCPlayStation 5Xbox Series X,
31,Lies of P,August 2023,lies-of-p,PCPlayStation 4+ 3 morePlayStation 5Xbox OneXb...,
2,Diablo IV,"Jun 6, 2023",diablo-iv,PCPlayStation 4+ 3 morePlayStation 5Xbox OneXb...,Return to Darkness with endless demons to slau...
20,Escape From Tarkov,Unknown Release Date,escape-from-tarkov,PC,Escape from Tarkov – a hardcore MMO mixture of...


In [ ]:
# add another website screening
other = []
# #url = f'https://www.gamespot.com/games/'{df.event}
url = 'https://www.digitaltrends.com/gaming/upcoming-ps5-games//'
result = requests.get(url, headers=headers)
soup = BeautifulSoup(result.content, "html.parser")
#print(result.content.decode())
#div class = "b-media__title b-media__title--small"
result = soup.find_all("div", {"class":"b-media__title b-media__title--small"})
for res in result : 
    other.append(res.text.strip())


In [ ]:
df1 = pd.DataFrame(list(other),
               columns =['other'])
df1['other1'] = df1['other'].str.split(' ').str.get(0)

In [ ]:
test = []
for x in df1['other1']:
    if df['event'].str.contains(x).any():
        test.append(None)
    else:
        test.append(x)
df1['test'] = test
df2 = df1.dropna()
df2['date'] = df2['other'].str.split(' - ').str.get(-1)
df2['event'] = df2['other'].str.split(' - ').str.get(0)
df2 = df2[df2['date'].str.contains(", 2023")]
df2 = df2[['event','date']]

/var/folders/jt/qpx02c196772_ytczgpv4llh0000gp/T/ipykernel_60207/3823963137.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['date'] = df2['other'].str.split(' - ').str.get(-1)
/var/folders/jt/qpx02c196772_ytczgpv4llh0000gp/T/ipykernel_60207/3823963137.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['event'] = df2['other'].str.split(' - ').str.get(0)


In [ ]:
df3 = df.append(df2)
df3

/var/folders/jt/qpx02c196772_ytczgpv4llh0000gp/T/ipykernel_60207/572790683.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df.append(df2)


,event,date,paged,platform,des
0,Resident Evil 4,"Mar 24, 2023",resident-evil-4,PCPlayStation 4+ 3 morePlayStation 5Xbox OneXb...,
1,The Legend of Zelda: Tears of the Kingdom,"May 12, 2023",the-legend-of-zelda-tears-of-the-kingdom,Nintendo Switch,
2,Diablo IV,"Jun 6, 2023",diablo-iv,PCPlayStation 4+ 3 morePlayStation 5Xbox OneXb...,Return to Darkness with endless demons to slau...
3,Final Fantasy XVI,"Jun 22, 2023",final-fantasy-xvi,PCPlayStation 5,
4,Star Wars Jedi: Survivor,"Apr 28, 2023",star-wars-jedi-survivor,PCPlayStation 5Xbox Series X,
5,Starfield,"Sep 6, 2023",starfield,PCXbox Series X,"Discover Starfield, the new space epic current..."
6,Street Fighter 6,"Jun 2, 2023",street-fighter-6,Arcade GamesPC+ 3 morePlayStation 4PlayStation...,
7,Dead Island 2,"Apr 21, 2023",dead-island-2,PCPlayStation 4+ 3 morePlayStation 5Xbox OneXb...,Dead Island 2 is an open world survival experi...
8,Dragon Age: Dreadwolf,Unknown Release Date,dragon-age-dreadwolf,PCPlayStation 4Xbox One,
9,Suicide Squad: Kill The Justice League,"May 26, 2023",suicide-squad-kill-the-justice-league,PCPlayStation 5Xbox Series X,


In [ ]:
df3.to_csv('events2023.csv')

In [ ]:
# for page in df.paged[:3]: 
#     url = f"https://www.gamespot.com/games/{page}"
#     print(url)
#     response = requests.get(url, headers=headers)
#     soup = BeautifulSoup(response.content, "html.parser")
#     plat = soup.find_all('ul',class_="game-module__platform")
#     for p in plat:
#         pi = p.text
#         platform.append(pi.split()[0])
#     desc = soup.find_all(class_="game-module__description")
#     for d in desc:
#         des.append(d.text)
#     print("page parsed")
#     sleep(randint(2,20))
    

In [26]:
jupyter nbconvert WSGamespot.ipynb --to script

SyntaxError: invalid syntax (690302667.py, line 1)